In [2]:
# Loading csvs
import pandas as pd
import numpy as np
from urllib.parse import urlparse

false = pd.read_csv('politifact_fake.csv', usecols=[1,2], skiprows=[0], na_values=['nan'], header=None, skip_blank_lines=True)
real = pd.read_csv('politifact_real.csv', usecols=[1,2], skiprows=[0], na_values=['nan'], header=None, skip_blank_lines=True)

# For some reason, the skip_blank_lines parameter wasn't working, so this is needed:
false.dropna(inplace=True)
real.dropna(inplace=True)

false_urls = false[[1]]
real_urls = real[[1]]

print(false.size)
false.head()


856


,1,2
0,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...
1,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...
2,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...
3,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide
4,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay


In [3]:
#Downloading text content from the URLs, and combining the text, URL, 
#and label into a single dataframe

from newspaper import Article

def url_preprocess(data, label):
    full_set = []
    #note: iterrows() is an antipattern, look into vectorizing?
    for i, row in data.iterrows():
        df_url = data.at[i, 1]
        
        if 'http://' not in df_url:
            if 'https://' not in df_url:
                df_url = 'https://' + df_url
            
        article = Article(url=df_url)
        
        try:
            article.download()
            article.parse()
            article_text = article.text
            
            cur = [df_url, data.at[i, 2], article_text, label]
            full_set.append(cur)
            
        except:
            article_text = 'ERROR'

            
    return full_set


In [4]:
false_list = url_preprocess(false, 0)
real_list = url_preprocess(real, 1)

df = pd.DataFrame(false_list, columns = ['url', 'title', 'text', 'label'])
df2 = pd.DataFrame(real_list, columns = ['url', 'title', 'text', 'label'])
df = df.append(df2, ignore_index=True)
df.to_csv('dataset.csv')

print(df)


                                                   url  \
0    https://www.nscdscamps.org/blog/category/paren...   
1    https://howafrica.com/oscar-pistorius-attempts...   
2    http://washingtonsources.org/trump-votes-for-d...   
3    https://gloria.tv/video/yRrtUtTCfPga6cq2VDJPcgQe4   
4    http://blogs.trendolizer.com/2015/01/new-york-...   
5    https://web.archive.org/web/20171027105356/htt...   
6    https://web.archive.org/web/20170702174006/htt...   
7    http://beforeitsnews.com/opinion-conservative/...   
8    https://web.archive.org/web/20170731052757/htt...   
9    https://jis.gov.jm/president-trump-underscores...   
10   https://me.me/i/actress-sandra-bullock-to-hill...   
11   http://therightists.com/gretchen-carlson-the-2...   
12   https://web.archive.org/web/20180503143610/htt...   
13   http://www.breitbart.com/california/2017/10/17...   
14   https://www.politico.com/story/2017/05/09/trum...   
15   https://www.mirror.co.uk/news/us-news/there-bl...   
16   https://w